In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
from IPython.display import HTML

In [2]:
IMAGE_SIZE = 256
CHANNELS = 3

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        horizontal_flip=True
)
train_generator = train_datagen.flow_from_directory(
        'PlantVillageDir/train',
        target_size=(IMAGE_SIZE,IMAGE_SIZE),
        batch_size=32,
        class_mode="sparse")

Found 1506 images belonging to 3 classes.


In [4]:
train_generator.class_indices

{'Potato___Early_blight': 0, 'Potato___Late_blight': 1, 'Potato___healthy': 2}

In [5]:
class_names = list(train_generator.class_indices.keys())
class_names

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

In [11]:
count=0
for image_batch, label_batch in train_generator:
    print(image_batch[0])
    print(image_batch.shape)
    print(label_batch[0])
    break

[[[0.69807744 0.62356764 0.6392539 ]
  [0.70587903 0.63136923 0.6470555 ]
  [0.70197314 0.62746334 0.6431496 ]
  ...
  [0.69411767 0.6313726  0.6313726 ]
  [0.69411767 0.6313726  0.6313726 ]
  [0.69411767 0.6313726  0.6313726 ]]

 [[0.6970439  0.6225341  0.63822037]
  [0.7051874  0.6306776  0.64636385]
  [0.70232403 0.62781423 0.6435005 ]
  ...
  [0.69411767 0.6313726  0.6313726 ]
  [0.69411767 0.6313726  0.6313726 ]
  [0.69422215 0.63147706 0.63147706]]

 [[0.6959913  0.6214815  0.63716775]
  [0.70448565 0.62997586 0.6456621 ]
  [0.7026749  0.6281651  0.6438514 ]
  ...
  [0.7038344  0.6410893  0.6410893 ]
  [0.70488703 0.64214194 0.64214194]
  [0.7059397  0.6431946  0.6431946 ]]

 ...

 [[0.559344   0.44169688 0.44954002]
  [0.5547825  0.43713543 0.44497856]
  [0.5502211  0.43257397 0.4404171 ]
  ...
  [0.5241058  0.40645874 0.41430187]
  [0.5694729  0.45182583 0.45966896]
  [0.6331173  0.5154702  0.52331334]]

 [[0.521139   0.40349194 0.41133508]
  [0.5228934  0.40524638 0.4130895 ]


In [12]:
validation_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        horizontal_flip=True)
validation_generator = validation_datagen.flow_from_directory(
        'PlantVillageDir/val',
        target_size=(IMAGE_SIZE,IMAGE_SIZE),
        batch_size=32,
        class_mode="sparse"
)

Found 430 images belonging to 3 classes.


In [13]:
test_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        horizontal_flip=True)

test_generator = test_datagen.flow_from_directory(
        'PlantVillageDir/test',
        target_size=(IMAGE_SIZE,IMAGE_SIZE),
        batch_size=32,
        class_mode="sparse"
)

Found 216 images belonging to 3 classes.


### Building the Model

In [14]:
input_shape = (IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 3

model = models.Sequential([
    layers.InputLayer(input_shape=input_shape),
    layers.Conv2D(32, kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 254, 254, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 64)        3

### Compiling the Model

In [16]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [18]:
history = model.fit(
    train_generator,
    steps_per_epoch=47,
    batch_size=32,
    validation_data=validation_generator,
    validation_steps=13,
    verbose=1,
    epochs=20,
)

Epoch 1/20
47/47 [==============================] - 63s 1s/step - loss: 0.9221 - accuracy: 0.4986 - val_loss: 0.8733 - val_accuracy: 0.5529
Epoch 2/20
47/47 [==============================] - 58s 1s/step - loss: 0.7733 - accuracy: 0.6594 - val_loss: 0.5669 - val_accuracy: 0.7644
Epoch 3/20
47/47 [==============================] - 59s 1s/step - loss: 0.4627 - accuracy: 0.8005 - val_loss: 0.3609 - val_accuracy: 0.8558
Epoch 4/20
47/47 [==============================] - 61s 1s/step - loss: 0.3796 - accuracy: 0.8433 - val_loss: 0.3251 - val_accuracy: 0.8918
Epoch 5/20
47/47 [==============================] - 60s 1s/step - loss: 0.2827 - accuracy: 0.8901 - val_loss: 0.3300 - val_accuracy: 0.8582
Epoch 6/20
47/47 [==============================] - 60s 1s/step - loss: 0.2351 - accuracy: 0.8976 - val_loss: 0.4156 - val_accuracy: 0.8438
Epoch 7/20
47/47 [==============================] - 72s 2s/step - loss: 0.1913 - accuracy: 0.9281 - val_loss: 0.2418 - val_accuracy: 0.9135
Epoch 8/20
47/47 [==

In [19]:
scores = model.evaluate(test_generator)

7/7 [==============================] - 3s 433ms/step - loss: 0.0995 - accuracy: 0.9815


In [20]:
scores

[0.09950443357229233, 0.9814814925193787]

#### Saving the Model

- Save model in h5 format so that there is just one file and we can upload that to GCP conveniently

In [21]:
model.save("../potatoes.h5")